In [1]:
#Import libraries
import math
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import re
import os.path
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError                                               

#Set pandas dataframe viewing options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
#Define function to scrape linkedin
def linkedin_scraper(url):

    #URL must be formatted without the job id in the URL and with '&start={}' at the end of the URL, allowing for the .format command to work
    chrome_driver.get(url.format(0))

    #Time.sleep is used to allow enough time for the browser to load
    time.sleep(5)

    #Locates the number of search results and converts this into how many pages there are for the search query
    noresults = int(''.join(re.findall(r'\d+',chrome_driver.find_element(By.XPATH,"//div[@class='jobs-search-results-list__subtitle']").text)))
    nopages = math.ceil(noresults/25)

    #Converts the current page into a beautiful soup object and extracts all the IDs (this is the only field not needing javascript scrolling
    soup = BeautifulSoup(chrome_driver.page_source,'html.parser')
    id = [x.get('data-occludable-job-id') for x in soup.find_all('li',attrs={'data-occludable-job-id':True})]
    titles = []
    companies = []
    links = []

    #Loops through the extracted IDs
    #Scrolls to the position of the current ID then extracts the title, company, and link at that location
    for x in id:
        chrome_driver.execute_script("arguments[0].scrollIntoView(true);",chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']"))
        titles = titles + [chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']//a[@class='disabled ember-view job-card-container__link job-card-list__title']").text]
        companies = companies + [chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']//span[@class='job-card-container__primary-description ']").text]
        links = links + [chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']//a[@class='disabled ember-view job-card-container__link job-card-list__title']").get_attribute('href')]

    #If there is more than 1 page of search results runs the below code
    if nopages > 1:
        
        #For each additional page, navigates to a new URL associated with the next page and extracts the data on that page
        for i in range(1,nopages):
            chrome_driver.get(url.format(i*25))
            time.sleep(5)
            soup = BeautifulSoup(chrome_driver.page_source,'html.parser')
            newid = [x.get('data-occludable-job-id') for x in soup.find_all('li',attrs={'data-occludable-job-id':True})]
            id = id + newid
            for x in newid:
                chrome_driver.execute_script("arguments[0].scrollIntoView(true);",chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']"))
                titles = titles + [chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']//a[@class='disabled ember-view job-card-container__link job-card-list__title']").text]
                companies = companies + [chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']//span[@class='job-card-container__primary-description ']").text]
                links = links + [chrome_driver.find_element(By.XPATH,f"//li[@data-occludable-job-id='{x}']//a[@class='disabled ember-view job-card-container__link job-card-list__title']").get_attribute('href')]

    #Combines the extracted data into a dataframe called 'linkedin'
    linkedin = pd.DataFrame({'ID':id,'Title':titles,'Company':companies,'Link':links})
    return linkedin

#Initialises the Selenium chrome driver. Options are not specified but can be added as needed (code does not work in headless mode)
chrome_options = webdriver.ChromeOptions()
chrome_driver = webdriver.Chrome(options=chrome_options)

#Navigates to the LinkedIn login page and sends credentials as specified here
chrome_driver.get("https://linkedin.com/uas/login")
time.sleep(5)
chrome_driver.find_element(By.ID,"username").send_keys('INPUT_LINKEDIN_USERNAME') 
chrome_driver.find_element(By.ID,"password").send_keys('INPUT_LINKEDIN_PASSWORD')
chrome_driver.find_element(By.XPATH,"//button[@type='submit']").click()
time.sleep(5)

#Runs LinkedIn scraping for four distinct keywords: 'Graduate Data', 'Intern Data', 'Graduate Economics', and 'Intern Economics'
#Each query is limited to the past 24 hours and with the experience levels 'Intern' and 'Entry Level'
grad_data = linkedin_scraper("https://www.linkedin.com/jobs/search/?distance=25&f_E=1%2C2&f_TPR=r86400&geoId=102257491&keywords=graduate%20data&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&start={}")
intern_data = linkedin_scraper("https://www.linkedin.com/jobs/search/?f_E=1%2C2&f_TPR=r86400&geoId=102257491&keywords=intern%20data&location=London%2C%20England%2C%20United%20Kingdom&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&start={}")
grad_econ = linkedin_scraper("https://www.linkedin.com/jobs/search/?f_E=1%2C2&f_TPR=r86400&geoId=102257491&keywords=graduate%20economics&location=London%2C%20England%2C%20United%20Kingdom&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start={}")
intern_econ = linkedin_scraper("https://www.linkedin.com/jobs/search/?f_E=1%2C2&f_TPR=r86400&geoId=102257491&keywords=intern%20economics&location=London%2C%20England%2C%20United%20Kingdom&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start={}")

#Combines the resulting dataframes into a final dataframe without duplicates
linkedin = pd.concat([grad_data,intern_data,grad_econ,intern_econ],axis=0,ignore_index=True)
linkedin.drop_duplicates(subset='ID',inplace=True)

In [3]:
#Define function to scrape Glassdoor
def glassdoor_scraper(url):
    
    #Opens up the query of interest
    chrome_driver.get(url)
    time.sleep(5)
    
    #Removes the cookie pop-up if there is one
    try:
        chrome_driver.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']").click()
        time.sleep(5)
    except:
        print('No pop-up')

    #Locates the number of search results and converts this into how many clicks are needed to display all the available results
    noresults = int(''.join(re.findall(r'\d+',chrome_driver.find_element(By.XPATH,"//h1[@data-test='search-title']").text)))
    noclicks = math.ceil(noresults/30)

    #If there is more than 1 page of search results runs the below code
    if noclicks > 1:

        #For each additional click, clicks the 'Load More' button at the bottom of the page
        for i in range(1,noclicks):
            chrome_driver.find_element(By.XPATH,"//button[@data-test='load-more']").click()
            time.sleep(5)

    #once all the results are displayed, converts the page to a beautiful soup object and extracts all the relevant information
    soup = BeautifulSoup(chrome_driver.page_source,'html.parser')
    links = [x.find('a')['href'] for x in soup.find('ul',attrs={'aria-label':'Jobs List'}).find_all('li')[:-1]]
    titles = [x.find('a').get_text() for x in soup.find('ul',attrs={'aria-label':'Jobs List'}).find_all('li')[:-1]]
    company = [x.find('span',attrs={'class':'EmployerProfile_employerName__Xemli'}).get_text() for x in soup.find('ul',attrs={'aria-label':'Jobs List'}).find_all('li')[:-1]]
    id = [x.get('data-jobid') for x in soup.find('ul',attrs={'aria-label':'Jobs List'}).find_all('li')[:-1]]

    #Saves the resulting lists into a dataframe
    glassdoor = pd.DataFrame({'ID':id,'Title':titles,'Company':company,'Link':links})
    return glassdoor

#Initial time.sleep is to allow space between the LinkedIn and Glassdoor scrapes
time.sleep(10)

#Navigates to the Glassdoor login page and logs in with Facebook using the below credentials
#Window switching is to ensure the relevant browser window is in focus
chrome_driver.get("https://www.glassdoor.co.uk/profile/login_input.htm")
time.sleep(5)
chrome_driver.find_element(By.XPATH,"//button[contains(@class, 'facebookWhite gd-btn center')]").click()
time.sleep(5)
chrome_driver.switch_to.window(chrome_driver.window_handles[-1])
chrome_driver.find_element(By.XPATH,"//input[contains(@name,'email')]").send_keys('INPUT_FACEBOOK_EMAIL')
chrome_driver.find_element(By.XPATH,"//input[contains(@name,'pass')]").send_keys('INPUT_FACEBOOK_PASSWORD')
chrome_driver.find_element(By.XPATH,"//input[contains(@name,'pass')]").send_keys(Keys.ENTER)
chrome_driver.switch_to.window(chrome_driver.window_handles[0])
time.sleep(15)

#Runs the same queries as LinkedIn
grad_data = glassdoor_scraper("https://www.glassdoor.co.uk/Job/london-england-graduate-data-jobs-SRCH_IL.0,14_IC2671300_KO15,28.htm?fromAge=3")
intern_data = glassdoor_scraper("https://www.glassdoor.co.uk/Job/london-england-intern-data-jobs-SRCH_IL.0,14_IC2671300_KO15,26.htm?fromAge=3")
grad_econ = glassdoor_scraper("https://www.glassdoor.co.uk/Job/london-england-graduate-economics-jobs-SRCH_IL.0,14_IC2671300_KO15,33.htm?fromAge=3")
intern_econ = glassdoor_scraper("https://www.glassdoor.co.uk/Job/london-england-intern-economics-jobs-SRCH_IL.0,14_IC2671300_KO15,31.htm?fromAge=3")

#Combines the dataframes into one big one without duplicates
glassdoor = pd.concat([grad_data,intern_data,grad_econ,intern_econ],axis=0,ignore_index=True)
glassdoor.drop_duplicates(subset='ID',inplace=True)

No pop-up
No pop-up
No pop-up


In [4]:
#Define function to scrape Indeed
def indeed_scraper(url):

    #Opens the query of interest
    chrome_driver.get(url)
    time.sleep(5)

    #Removes any pop-ups
    try:
        chrome_driver.find_element(By.XPATH,"//button[@id='onetrust-reject-all-handler']").click()
        time.sleep(3)
    except:
        print('No pop-up')
    try:
        chrome_driver.find_element(By.XPATH,"//button[@onclick='closeGoogleOnlyModal()']").click()
        time.sleep(3)
        chrome_driver.find_element(By.XPATH,"//button[@aria-label='close']").click()
        time.sleep(3)
    except:
        print('No pop-up')

    #Converts the current page to a beautiful soup object and exxtracts all relevant data
    soup = BeautifulSoup(chrome_driver.page_source,'html.parser')
    id = [item for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'}) for item in x.find('div').get('class') if 'job' in item]
    titles = [x.find('h2').get_text() if x.find('h2') is not None else '-' for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'})]
    titles = [x for x in titles if x != '-']
    company = [x.get_text() for x in soup.find_all('span',attrs={'data-testid':'company-name'})]
    link = ['https://uk.indeed.com' + x.find('a').get('href') if x.find('a') is not None else '-' for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'})]
    link = [x for x in link if x != '-']

    #If a button to navigate to the next page of results exists, extract the data from the next pages
    if soup.find('ul',attrs={'class':'css-1g90gv6 eu4oa1w0'}).find('a',attrs={'aria-label':'Next Page'}) is not None:
        chrome_driver.find_element(By.XPATH,"//a[@aria-label='Next Page']").click()
        time.sleep(5)
        try:
            chrome_driver.find_elementz(By.XPATH,"//button[@onclick='closeGoogleOnlyModal()']").click()
            time.sleep(3)
            chrome_driver.find_element(By.XPATH,"//button[@aria-label='close']").click()
            time.sleep(3)
        except:
            print('No pop-up')
        soup = BeautifulSoup(chrome_driver.page_source,'html.parser')
        id = id + [item for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'}) for item in x.find('div').get('class') if 'job' in item]
        titles = titles + [x.find('h2').get_text() if x.find('h2') is not None else '-' for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'})]
        titles = [x for x in titles if x != '-']
        company = company + [x.get_text() for x in soup.find_all('span',attrs={'data-testid':'company-name'})]
        link = link + ['https://uk.indeed.com' + x.find('a').get('href') if x.find('a') is not None else '-' for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'})]
        link = [x for x in link if x != '-']

    #If, after the first next page press, there are still more pages, the code will keep extracting data from the next page until the next page button no longer exists
    while soup.find('ul',attrs={'class':'css-1g90gv6 eu4oa1w0'}).find('a',attrs={'aria-label':'Next Page'}) is not None:
        chrome_driver.find_element(By.XPATH,"//a[@aria-label='Next Page']").click()
        time.sleep(5)
        soup = BeautifulSoup(chrome_driver.page_source,'html.parser')
        id = id + [item for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'}) for item in x.find('div').get('class') if 'job' in item]
        titles = titles + [x.find('h2').get_text() if x.find('h2') is not None else '-' for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'})]
        titles = [x for x in titles if x != '-']
        company = company + [x.get_text() for x in soup.find_all('span',attrs={'data-testid':'company-name'})]
        link = link + ['https://uk.indeed.com' + x.find('a').get('href') if x.find('a') is not None else '-' for x in soup.find_all('li',attrs={'class':'css-5lfssm eu4oa1w0'})]
        link = [x for x in link if x != '-']

    #Saves the data as a dataframe
    indeed = pd.DataFrame({'ID':id,'Title':titles,'Company':company,'Link':link})
    return indeed

#Time.sleep to give enough time between Glassdoor and Indeed
time.sleep(10)

#Runs the same queries but on Indeed
grad_data = indeed_scraper("https://uk.indeed.com/jobs?q=graduate+data&l=London%2C+Greater+London&fromage=1")
intern_data = indeed_scraper("https://uk.indeed.com/jobs?q=intern+data&l=London%2C+Greater+London&fromage=1")
grad_econ = indeed_scraper("https://uk.indeed.com/jobs?q=graduate+economics&l=London%2C+Greater+London&fromage=1")
intern_econ = indeed_scraper("https://uk.indeed.com/jobs?q=intern+economics&l=London%2C+Greater+London&fromage=1")

#Saves the outputs into one dataframe without duplicates
indeed = pd.concat([grad_data,intern_data,grad_econ,intern_econ],axis=0,ignore_index=True)
indeed.drop_duplicates(subset='ID',inplace=True)

#Closes the chrome driver
chrome_driver.quit()

No pop-up
No pop-up
No pop-up
No pop-up
No pop-up
No pop-up
No pop-up


In [5]:
#Activates the Google Sheets API credentials
#Requires a Google Cloud account and for a project to be open with OAuth credentials
def credentials():
    scopes = "https://www.googleapis.com/auth/spreadsheets"
    credentials = None
    if os.path.exists("API_Files/token.json"):
        credentials = Credentials.from_authorized_user_file("API_Files/token.json",scopes)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("API_Files/credentials.json",scopes)
            credentials = flow.run_local_server(port=0)
        with open("API_Files/token.json","w") as token:
            token.write(credentials.to_json())
    return credentials
credentials = credentials()

#Builds the Google Sheets API service
service = build("sheets","v4",credentials=credentials)

#For each data source this is the order of operations
# 1. Pull existing data from the relevant tab in the Google Sheet
# 2. Filters the scraped data to only include the data not already on the Google Sheet (uses the 'ID' column)
# 3. Concatenates the existing data with the new data into one dataframe, ready to be uploaded
# 4. Updates the relevant tabs with the complete dataframes in the Google Sheet

request = service.spreadsheets().values().get(spreadsheetId="SPREADSHEET_ID",range='LinkedIn').execute()
current_linkedin = pd.DataFrame(data=request['values'][1:],columns=request['values'][0])
linkedin_to_upload = linkedin.loc[~linkedin['ID'].isin(current_linkedin['ID'])]
linkedin = pd.concat([current_linkedin,linkedin_to_upload],axis=0,ignore_index=True)
body = {'values':[linkedin.columns.tolist()] + linkedin.values.tolist()}
request = service.spreadsheets().values().update(spreadsheetId="SPREADSHEET_ID",range='LinkedIn',valueInputOption="USER_ENTERED",body=body).execute()

request = service.spreadsheets().values().get(spreadsheetId="SPREADSHEET_ID",range='Glassdoor').execute()
current_glassdoor = pd.DataFrame(data=request['values'][1:],columns=request['values'][0])
glassdoor_to_upload = glassdoor.loc[~glassdoor['ID'].isin(current_glassdoor['ID'])]
glassdoor = pd.concat([current_glassdoor,glassdoor_to_upload],axis=0,ignore_index=True)
body = {'values':[glassdoor.columns.tolist()] + glassdoor.values.tolist()}
request = service.spreadsheets().values().update(spreadsheetId="SPREADSHEET_ID",range='Glassdoor',valueInputOption="USER_ENTERED",body=body).execute()

request = service.spreadsheets().values().get(spreadsheetId="SPREADSHEET_ID",range='Indeed').execute()
current_indeed = pd.DataFrame(data=request['values'][1:],columns=request['values'][0])
indeed_to_upload = indeed.loc[~indeed['ID'].isin(current_indeed['ID'])]
indeed = pd.concat([current_indeed,indeed_to_upload],axis=0,ignore_index=True)
body = {'values':[indeed.columns.tolist()] + indeed.values.tolist()}
request = service.spreadsheets().values().update(spreadsheetId="SPREADSHEET_ID",range='Indeed',valueInputOption="USER_ENTERED",body=body).execute()